# Workflow s lidským zapojením pomocí Microsoft Agent Framework

## 🎯 Cíle učení

V tomto notebooku se naučíte, jak implementovat workflow s **lidským zapojením** pomocí `RequestInfoExecutor` v Microsoft Agent Framework. Tento výkonný vzor umožňuje pozastavit AI workflow za účelem získání lidského vstupu, čímž se agenti stávají interaktivními a lidé získávají kontrolu nad klíčovými rozhodnutími.

## 🔄 Co je lidské zapojení?

**Lidské zapojení (HITL)** je návrhový vzor, kdy AI agenti pozastaví provádění, aby požádali o lidský vstup před pokračováním. To je zásadní pro:

- ✅ **Klíčová rozhodnutí** - Získání lidského schválení před provedením důležitých akcí
- ✅ **Nejasné situace** - Umožnění lidem objasnit, když si AI není jistá
- ✅ **Uživatelské preference** - Dotazování uživatelů na výběr mezi více možnostmi
- ✅ **Soulad a bezpečnost** - Zajištění lidského dohledu u regulovaných operací
- ✅ **Interaktivní zážitky** - Vytváření konverzačních agentů, kteří reagují na uživatelský vstup

## 🏗️ Jak to funguje v Microsoft Agent Framework

Framework poskytuje tři klíčové komponenty pro HITL:

1. **`RequestInfoExecutor`** - Speciální executor, který pozastaví workflow a vydá `RequestInfoEvent`
2. **`RequestInfoMessage`** - Základní třída pro typizované požadavky zasílané lidem
3. **`RequestResponse`** - Spojuje lidské odpovědi s původními požadavky pomocí `request_id`

**Vzor workflow:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Náš příklad: Rezervace hotelu s potvrzením uživatele

Navážeme na podmíněný workflow přidáním lidského potvrzení **předtím**, než navrhneme alternativní destinace:

1. Uživatel požádá o destinaci (např. "Paříž")
2. `availability_agent` zkontroluje dostupnost pokojů
3. **Pokud nejsou pokoje dostupné** → `confirmation_agent` se zeptá "Chcete vidět alternativy?"
4. Workflow se **pozastaví** pomocí `RequestInfoExecutor`
5. **Člověk odpoví** "ano" nebo "ne" prostřednictvím konzolového vstupu
6. `decision_manager` rozhodne na základě odpovědi:
   - **Ano** → Zobrazí alternativní destinace
   - **Ne** → Zruší požadavek na rezervaci
7. Zobrazí se konečný výsledek

Tímto způsobem umožníte uživatelům kontrolovat návrhy agenta!

---

Pojďme začít! 🚀


## Krok 1: Importujte potřebné knihovny

Importujeme standardní komponenty Agent Frameworku plus **specifické třídy pro zapojení člověka do procesu**:
- `RequestInfoExecutor` - Executor, který pozastaví workflow pro vstup od člověka
- `RequestInfoEvent` - Událost vyvolaná při požadavku na vstup od člověka
- `RequestInfoMessage` - Základní třída pro typizované datové požadavky
- `RequestResponse` - Spojuje odpovědi od člověka s požadavky
- `WorkflowOutputEvent` - Událost pro detekci výstupů workflowu


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Krok 2: Definujte Pydantic modely pro strukturované výstupy

Tyto modely definují **schéma**, které agenti budou vracet. Zachováváme všechny modely z podmíněného pracovního postupu a přidáváme:

**Nové pro zapojení člověka do procesu:**
- `HumanFeedbackRequest` - Podtřída `RequestInfoMessage`, která definuje požadavek odesílaný lidem
  - Obsahuje `prompt` (otázku k položení) a `destination` (kontext o nedostupném městě)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Krok 3: Vytvoření nástroje pro rezervaci hotelu

Stejný nástroj jako v podmínkovém pracovním postupu – kontroluje, zda jsou v cílové destinaci dostupné pokoje.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Krok 4: Definujte podmínkové funkce pro směrování

Potřebujeme **čtyři podmínkové funkce** pro náš workflow s lidským zapojením:

**Z podmínkového workflow:**
1. `has_availability_condition` - Směruje, když hotely JSOU dostupné
2. `no_availability_condition` - Směruje, když hotely NEJSOU dostupné

**Nové pro workflow s lidským zapojením:**
3. `user_wants_alternatives_condition` - Směruje, když uživatel řekne "ano" na alternativy
4. `user_declines_alternatives_condition` - Směruje, když uživatel řekne "ne" na alternativy


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            wants_alternatives = "wants to see alternative" in msg_text or "want to see alternative" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                    <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{wants_alternatives}</strong>
                </div>
            """)
            )
            
            return wants_alternatives
    
    return False
def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorRequest sent by decision_manager.
    """
    # Check if it's an AgentExecutorRequest (what decision_manager sends)
    if isinstance(message, AgentExecutorRequest):
        # Check the message text to determine user's choice
        if message.messages and len(message.messages) > 0:
            msg_text = message.messages[0].text.lower()
            declined = "declined" in msg_text or "has declined" in msg_text
            
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{declined}</strong>
                </div>
            """)
            )
            
            return declined
    
    return False
print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Krok 5: Vytvoření Decision Manager Executor

Toto je **jádro vzoru "člověk v procesu rozhodování"!** `DecisionManager` je vlastní `Executor`, který:

1. **Přijímá zpětnou vazbu od člověka** prostřednictvím objektů `RequestResponse`
2. **Zpracovává rozhodnutí uživatele** (ano/ne)
3. **Směruje pracovní postup** zasíláním zpráv příslušným agentům

Klíčové vlastnosti:
- Používá dekorátor `@handler` k zpřístupnění metod jako kroků pracovního postupu
- Přijímá `RequestResponse[HumanFeedbackRequest, str]`, který obsahuje jak původní požadavek, tak odpověď uživatele
- Vytváří jednoduché zprávy "ano" nebo "ne", které spouštějí naše podmínkové funkce


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and let the workflow route based on conditions.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        This handler just displays feedback and passes the RequestResponse through.
        The routing is done by condition functions on the edges.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Will route to alternative_agent
                </div>
            """)
            )
            # Create and send a message for the alternative_agent
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Will route to cancellation_agent
                </div>
            """)
            )
            # Create and send a message for the cancellation_agent
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input - treat as decline
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(
                Role.USER,
                text="The user has declined to see alternatives. Please acknowledge their decision.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Krok 6: Vytvoření vlastního vykonavatele zobrazení

Stejný vykonavatel zobrazení z podmíněného pracovního postupu - poskytuje konečné výsledky jako výstup pracovního postupu.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Krok 7: Načtení proměnných prostředí

Nakonfigurujte klienta LLM (GitHub Models, Azure OpenAI nebo OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Krok 8: Vytvoření AI agentů a vykonavatelů

Vytvoříme **šest komponent workflow**:

**Agenti (zabalení v AgentExecutor):**
1. **availability_agent** - Kontroluje dostupnost hotelů pomocí nástroje
2. **confirmation_agent** - 🆕 Připravuje žádost o potvrzení od člověka
3. **alternative_agent** - Navrhuje alternativní města (když uživatel řekne ano)
4. **booking_agent** - Podporuje rezervaci (když jsou pokoje dostupné)
5. **cancellation_agent** - 🆕 Zpracovává zprávu o zrušení (když uživatel řekne ne)

**Speciální vykonavatelé:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor`, který pozastaví workflow pro vstup od člověka
7. **decision_manager** - 🆕 Vlastní vykonavatel, který směruje na základě odpovědi od člověka (již definováno výše)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Krok 9: Vytvoření pracovního postupu s lidským zapojením

Nyní vytvoříme graf pracovního postupu s **podmíněným směrováním**, včetně cesty s lidským zapojením:

**Struktura pracovního postupu:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Klíčové hrany:**
- `availability_agent → confirmation_agent` (když nejsou dostupné pokoje)
- `confirmation_agent → prepare_human_request` (transformace typu)
- `prepare_human_request → request_info_executor` (pauza pro člověka)
- `request_info_executor → decision_manager` (vždy - poskytuje RequestResponse)
- `decision_manager → alternative_agent` (když uživatel řekne "ano")
- `decision_manager → cancellation_agent` (když uživatel řekne "ne")
- `availability_agent → booking_agent` (když jsou pokoje dostupné)
- Všechny cesty končí u `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Krok 10: Spuštění testovacího případu 1 - Město BEZ dostupnosti (Paříž s lidským potvrzením)

Tento test ukazuje **celý cyklus s lidským zapojením**:

1. Požadavek na hotel v Paříži
2. kontrola availability_agent → Žádné pokoje
3. confirmation_agent vytvoří otázku pro člověka
4. request_info_executor **pozastaví workflow** a vydá `RequestInfoEvent`
5. **Aplikace detekuje událost a vyzve uživatele v konzoli**
6. Uživatel zadá "ano" nebo "ne"
7. Aplikace odešle odpověď prostřednictvím `send_responses_streaming()`
8. decision_manager rozhodne na základě odpovědi
9. Zobrazení konečného výsledku

**Klíčový vzor:**
- Použijte `workflow.run_stream()` pro první iteraci
- Použijte `workflow.send_responses_streaming(pending_responses)` pro následující iterace
- Poslouchejte `RequestInfoEvent`, abyste zjistili, kdy je potřeba lidský vstup
- Poslouchejte `WorkflowOutputEvent`, abyste zachytili konečné výsledky


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Krok 11: Spuštění testovacího případu 2 - Město S dostupností (Stockholm - Není potřeba lidský vstup)

Tento test ukazuje **přímou cestu**, když jsou pokoje dostupné:

1. Požadavek na hotel ve Stockholmu  
2. availability_agent kontroluje → Pokoje dostupné ✅  
3. booking_agent navrhuje rezervaci  
4. display_result zobrazí potvrzení  
5. **Není potřeba žádný lidský vstup!**

Pracovní postup zcela obchází cestu s lidským zapojením, pokud jsou pokoje dostupné.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Klíčové poznatky a osvědčené postupy pro zapojení člověka do procesu

### ✅ Co jste se naučili:

#### 1. **RequestInfoExecutor Pattern**
Vzor zapojení člověka do procesu v Microsoft Agent Framework využívá tři klíčové komponenty:
- `RequestInfoExecutor` - Pozastaví workflow a generuje události
- `RequestInfoMessage` - Základní třída pro typizované požadavky (vytvořte podtřídu!)
- `RequestResponse` - Spojuje odpovědi od lidí s původními požadavky

**Klíčové pochopení:**
- `RequestInfoExecutor` sám o sobě nesbírá vstupy - pouze pozastaví workflow
- Vaše aplikační kód musí poslouchat `RequestInfoEvent` a sbírat vstupy
- Musíte zavolat `send_responses_streaming()` s dict mapujícím `request_id` na odpověď uživatele

#### 2. **Vzor streamovaného provádění**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Architektura založená na událostech**
Poslouchejte specifické události pro řízení workflow:
- `RequestInfoEvent` - Je potřeba vstup od člověka (workflow pozastaven)
- `WorkflowOutputEvent` - Konečný výsledek je dostupný (workflow dokončen)
- `WorkflowStatusEvent` - Změny stavu (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, atd.)

#### 4. **Vlastní executory s @handler**
`DecisionManager` ukazuje, jak vytvořit executory, které:
- Používají dekorátor `@handler` k vystavení metod jako kroků workflow
- Přijímají typizované zprávy (např. `RequestResponse[HumanFeedbackRequest, str]`)
- Směřují workflow zasíláním zpráv jiným executorům
- Přistupují ke kontextu přes `WorkflowContext`

#### 5. **Podmíněné směrování na základě rozhodnutí člověka**
Můžete vytvořit funkce podmínek, které vyhodnocují odpovědi od lidí:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Reálné aplikace:

1. **Schvalovací workflow**
   - Získání schválení od manažera před zpracováním výdajových zpráv
   - Vyžadování lidské kontroly před odesláním automatizovaných e-mailů
   - Potvrzení transakcí s vysokou hodnotou před provedením

2. **Moderace obsahu**
   - Označení sporného obsahu pro lidskou kontrolu
   - Požádání moderátorů o konečné rozhodnutí v hraničních případech
   - Eskalace na člověka, když je důvěra AI nízká

3. **Zákaznický servis**
   - Nechat AI automaticky řešit rutinní dotazy
   - Eskalace složitých problémů na lidské agenty
   - Zeptat se zákazníka, zda chce mluvit s člověkem

4. **Zpracování dat**
   - Požádat lidi o vyřešení nejasných datových záznamů
   - Potvrdit interpretace AI u nejasných dokumentů
   - Nechat uživatele vybrat mezi několika platnými interpretacemi

5. **Systémy kritické pro bezpečnost**
   - Vyžadovat lidské potvrzení před nevratnými akcemi
   - Získat schválení před přístupem k citlivým datům
   - Potvrdit rozhodnutí v regulovaných odvětvích (zdravotnictví, finance)

6. **Interaktivní agenti**
   - Vytvořit konverzační boty, kteří kladou doplňující otázky
   - Navrhnout průvodce, kteří uživatele provádějí složitými procesy
   - Navrhnout agenty, kteří spolupracují s lidmi krok za krokem

### 🔄 Porovnání: S vs Bez zapojení člověka do procesu

| Funkce | Podmíněný workflow | Workflow se zapojením člověka |
|--------|---------------------|------------------------------|
| **Provádění** | Jedno `workflow.run()` | Smyčka s `run_stream()` + `send_responses_streaming()` |
| **Vstup uživatele** | Žádný (plně automatizovaný) | Interaktivní výzvy přes `input()` nebo UI |
| **Komponenty** | Agenti + Executory | + RequestInfoExecutor + DecisionManager |
| **Události** | Pouze AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, atd. |
| **Pozastavení** | Bez pozastavení | Workflow se pozastaví u RequestInfoExecutor |
| **Kontrola člověkem** | Žádná kontrola člověkem | Lidé dělají klíčová rozhodnutí |
| **Použití** | Automatizace | Spolupráce a dohled |

### 🚀 Pokročilé vzory:

#### Více bodů rozhodování člověkem
Můžete mít více uzlů `RequestInfoExecutor` ve stejném workflow:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Zpracování časových limitů
Implementujte časové limity pro odpovědi od lidí:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integrace s bohatým UI
Namísto `input()` integrujte s webovým UI, Slackem, Teams, atd.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Podmíněné zapojení člověka do procesu
Požádejte o vstup od člověka pouze v konkrétních situacích:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Osvědčené postupy:

1. **Vždy vytvořte podtřídu RequestInfoMessage**
   - Poskytuje typovou bezpečnost a validaci
   - Umožňuje bohatý kontext pro vykreslení UI
   - Zjasňuje záměr každého typu požadavku

2. **Používejte popisné výzvy**
   - Zahrňte kontext o tom, na co se ptáte
   - Vysvětlete důsledky každé volby
   - Udržujte otázky jednoduché a jasné

3. **Zpracovávejte neočekávané vstupy**
   - Validujte odpovědi uživatelů
   - Poskytněte výchozí hodnoty pro neplatné vstupy
   - Dejte jasné chybové zprávy

4. **Sledujte ID požadavků**
   - Používejte korelaci mezi request_id a odpověďmi
   - Nesnažte se spravovat stav ručně

5. **Navrhujte pro neblokování**
   - Neblokujte vlákna čekáním na vstup
   - Používejte asynchronní vzory
   - Podporujte současné instance workflow

### 📚 Související koncepty:

- **Agent Middleware** - Zachytávání volání agentů (předchozí notebook)
- **Správa stavu workflow** - Ukládání stavu workflow mezi běhy
- **Spolupráce více agentů** - Kombinace zapojení člověka do procesu s týmy agentů
- **Architektury založené na událostech** - Budování reaktivních systémů s událostmi

---

### 🎓 Gratulujeme!

Ovládli jste workflow se zapojením člověka do procesu v Microsoft Agent Framework! Nyní víte, jak:
- ✅ Pozastavit workflow pro získání vstupu od člověka
- ✅ Používat RequestInfoExecutor a RequestInfoMessage
- ✅ Zpracovávat streamované provádění pomocí událostí
- ✅ Vytvářet vlastní executory s @handler
- ✅ Směrovat workflow na základě rozhodnutí člověka
- ✅ Budovat interaktivní AI agenty, kteří spolupracují s lidmi

**Toto je klíčový vzor pro budování důvěryhodných a kontrolovatelných AI systémů!** 🚀



---

**Prohlášení**:  
Tento dokument byl přeložen pomocí služby AI pro překlady [Co-op Translator](https://github.com/Azure/co-op-translator). Ačkoli se snažíme o přesnost, mějte na paměti, že automatické překlady mohou obsahovat chyby nebo nepřesnosti. Původní dokument v jeho rodném jazyce by měl být považován za autoritativní zdroj. Pro důležité informace se doporučuje profesionální lidský překlad. Neodpovídáme za žádná nedorozumění nebo nesprávné interpretace vyplývající z použití tohoto překladu.
